In [17]:
import requests
import lxml
from bs4 import BeautifulSoup
from collections import defaultdict
import pandas as pd

In [18]:
def format(string):
    return str(string).zfill(6)

company_data = pd.read_csv('data_0916_20230808.csv', encoding='cp949')
company_data['종목코드'] = company_data['종목코드'].apply(format)


In [19]:
num_company = list(company_data['종목코드'])
num_company

['060310',
 '095570',
 '006840',
 '054620',
 '265520',
 '211270',
 '027410',
 '282330',
 '126600',
 '138930',
 '001460',
 '001465',
 '013720',
 '083790',
 '001040',
 '079160',
 '035760',
 '311690',
 '00104K',
 '000120',
 '011150',
 '011155',
 '001045',
 '097950',
 '097955',
 '051500',
 '058820',
 '023460',
 '056730',
 '000480',
 '065770',
 '083660',
 '000590',
 '012030',
 '404950',
 '456440',
 '367360',
 '016610',
 '005830',
 '000990',
 '139130',
 '060900',
 '025440',
 '001530',
 '000210',
 '001880',
 '000215',
 '375500',
 '37550L',
 '37550K',
 '068790',
 '007340',
 '004840',
 '241520',
 '065150',
 '155660',
 '069730',
 '180400',
 '017940',
 '245620',
 '037370',
 '278990',
 '365550',
 '050120',
 '383220',
 '007700',
 '214270',
 '130500',
 '114090',
 '900290',
 '078930',
 '083450',
 '006360',
 '001250',
 '007070',
 '078935',
 '297890',
 '078150',
 '012630',
 '039570',
 '089470',
 '294870',
 '009540',
 '267250',
 '267270',
 '322000',
 '042670',
 '267260',
 '329180',
 '097230',
 '195940',

In [20]:
finance_dict = defaultdict(list)
com=[]
com2 = []

In [21]:
for nc in num_company:
    finance_dict['num_company'].append(nc)
    one_company_volume = []
    one_company_updown = []
    for j in range(1,3):
        url = f"https://finance.naver.com/item/frgn.naver?code={nc}&page={j}"
        try:
            response = requests.get(url, headers={'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'})
            soup = BeautifulSoup(response.text, "lxml")
            elements_with_class = soup.select("#content > div.section.inner_sub > table.type2 > tr")
            out = [5,6,7,13,14,15,21,22,23,29,30,31]
            trading_volumes =[]
            updown = []
        except:
            if j == 1:
                finance_dict['trading_volume'].append(None)
                finance_dict['updown'].append(None)
            continue
        try:
            num_all_stock_p = soup.select('#tab_con1 > div.first > table > tr')[2]
            num_all_stock = num_all_stock_p.select('td > em')[0].text
        except:
            if j == 1:
                finance_dict['all_stock'].append(None)
            continue
        #print(num_all_stock) # 상장주식수
        if j == 1:
            finance_dict['all_stock'].append(num_all_stock)

        for i in range(32): 
            if i in out:
                continue
            if j==2 and len(trading_volumes) >= 4:
                continue
            minus_plus = elements_with_class[i+3].find_all('td')[3].text
            if '-' in minus_plus:
                status = 0
            elif '+' in minus_plus:
                status = 1
            else:
                status = -1
            updown.append(status)
            trading_volume = elements_with_class[i+3].find_all('td')[4].text
            trading_volumes.append(trading_volume)
        #print(trading_volumes)
        one_company_volume.append(trading_volumes)
        one_company_updown.append(updown)
    com2.append(one_company_updown)
    com.append(one_company_volume)
    try:
        m = one_company_volume[0]+one_company_volume[1]
        n = one_company_updown[0]+one_company_updown[1]
    except:
        finance_dict['trading_volume'].append(None)
        finance_dict['updown'].append(None)
        continue

    m = [w.replace(',', '') for w in m]
    finance_dict['trading_volume'].append(m)
    finance_dict['updown'].append(n)
# for c1,c2 in zip(com,com2):
#     try:
#         m = c1[0]+c1[1]
#         n = c2[0]+c2[1]
#     except:
#         continue
#     # print(i)
#     m = [w.replace(',', '') for w in m]
#     finance_dict['trading_volume'].append(m)
#     finance_dict['updown'].append(n)

In [22]:
finance_dict['updown']

[[1, 0, 1, 0, -1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0],
 [1, 1, 1, 1, 0, 1, 1, -1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, -1, -1, 1, -1, 1, 0],
 [0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, -1, 1, 1, 1, 1, 1, 1, 0, 0, -1, 0],
 [0, -1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0],
 [0, -1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0],
 [0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
 [1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, -1, 0, 1, 0, 0, 0],
 [1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0],
 [0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, -1, 0, 0, 0, -1, 0, 1, 1, 0, 0],
 [1, -1, 1, 0, 0, 1, 1, 0, 1, -1, 0, 0, 1, 1, 0, 0, -1, 1, 1, 1, 1, 0, 0, 0],
 [0, 1, 1, 0, 0, 1, 1, -1, 0, -1, 1, 1, -1, 0, -1, 0, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, -1, 1, 0, -1, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, -1, 1],
 [1

In [23]:
com2

[[[1, 0, 1, 0, -1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1], [0, 1, 0, 0]],
 [[1, 1, 1, 1, 0, 1, 1, -1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, -1, -1],
  [1, -1, 1, 0]],
 [[0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, -1, 1, 1, 1, 1, 1, 1],
  [0, 0, -1, 0]],
 [[0, -1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0], [0, 1, 0, 0]],
 [[0, -1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1], [1, 0, 0, 0]],
 [[0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [1, 0, 0, 0]],
 [[1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, -1, 0], [1, 0, 0, 0]],
 [[1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0], [0, 1, 1, 0]],
 [[0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, -1, 0, 0, 0, -1, 0],
  [1, 1, 0, 0]],
 [[1, -1, 1, 0, 0, 1, 1, 0, 1, -1, 0, 0, 1, 1, 0, 0, -1, 1, 1, 1],
  [1, 0, 0, 0]],
 [[0, 1, 1, 0, 0, 1, 1, -1, 0, -1, 1, 1, -1, 0, -1, 0, 1, 1, 1, 1],
  [1, 1, 1, 1]],
 [[1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, -1, 1, 0, -1, 0, 0, 1, 0],
  [0, 0, 0, 0]],
 [[0, 1, 1, 0, 0, 1

In [24]:
df = pd.DataFrame(finance_dict)

df.to_csv('company_data.csv', index=False)